In [ ]:
import numpy as np
import math
from itertools import combinations_with_replacement 

## Exploration

In [ ]:
rolls = np.zeros(shape=(6**6,6))
i=0
for a in range(1,7):
    for b in range(1,7):
        for c in range(1,7):
            for d in range(1,7):
                for e in range(1,7):
                    for f in range(1,7):
                        rolls[i] = np.array([a, b, c, d, e, f])
                        i+=1
np.savetxt("rolls.csv", rolls, delimiter=",")

In [ ]:
dice_dim = 3
num_dice = 4
num_unique_rolls = int((np.math.factorial(num_dice + dice_dim - 1)) / 
                       (np.math.factorial(num_dice) * np.math.factorial(dice_dim - 1)))

unique_rolls = np.ones(shape=(num_unique_rolls, num_dice))
i = 0

for a in range(dice_dim):
    for b in range(a, dice_dim):
        for c in range(b, dice_dim):
            for d in range(c, dice_dim):
                unique_rolls[i] += np.array([a, b, c, d])
                i += 1
unique_rolls

In [ ]:
if False:
    # What a waste of time
    # Rule 34 of Python: If a problem exists, there's already a library for it
    def generate_unique_rolls(dice_dim, num_dice):
        num_unique_rolls = int((np.math.factorial(num_dice + dice_dim - 1)) / 
                            (np.math.factorial(num_dice) * np.math.factorial(dice_dim - 1)))
        unique_rolls = np.zeros(shape=(num_unique_rolls, num_dice + 1))

        def roll_dice(current_roll, num_dice, start_index, unique_rolls, i):
            if num_dice == 0:
                current_roll = np.ones(6) + current_roll
                points = calculate_points(current_roll)
                unique_rolls[i] += np.append(current_roll, points)
                return i + 1

            for j in range(start_index, dice_dim):
                new_roll = current_roll + [j]
                i = roll_dice(new_roll, num_dice - 1, j, unique_rolls, i)

            return i

        roll_dice([], num_dice, 0, unique_rolls, 0)
        return unique_rolls

    dice_dim = 6
    num_dice = 6
    unique_rolls = generate_unique_rolls(dice_dim, num_dice)
    np.savetxt("unique_rolls.csv", unique_rolls, fmt='%d', delimiter=",")


## Helper Functions

In [ ]:
def calculate_points(roll):
    roll = np.array(roll).astype(int)
    counts = np.zeros(7)
    raw_counts = np.bincount(roll)
    counts[:len(raw_counts)] += raw_counts
    
    points = 0

    # Check for six of a kind
    for i in range(1, 7):
        if counts[i] == 6:
            return 3000
        
    # Check for 1-6 straight
    if np.all(counts[1:] == 1):
        return 1500
    
    # Check for 3 pairs
    if np.sum(counts == 2) == 3:
        return 1500
    
    # Check for 2 triplets
    if np.sum(counts == 3) == 2:
        return 2500

    # Check for five of a kind
    for i in range(1, 7):
        if counts[i] == 5:
            points += 2000

            # Check remaining points
            if counts[1] == 1: points += 100
            if counts[5] == 1: points += 50

            return points

    # Check for four of a kind
    for i in range(1, 7):
        if counts[i] == 4:
            points += 1000

            # Check for pair
            for i in range(1, 7):
                if counts[i] == 2:
                    points += 500
                    return points
            
            # Check remaining points
            if counts[1] == 1: points += 100
            if counts[5] == 1: points += 50

            return points
    
    # Check for three of a kind
    for i in range(1, 7):
        if counts[i] == 3:
            if i == 1: points += 300
            else: points += i * 100

            # Check remaining points
            if counts[1] == 1: points += 100
            if counts[1] == 2: points += 200
            if counts[5] == 1: points += 50
            if counts[5] == 2: points += 100

            return points
    
    # Check remaining points
    if counts[1] == 1: points += 100
    if counts[1] == 2: points += 200
    if counts[5] == 1: points += 50
    if counts[5] == 2: points += 100

    return points
    

# Test the function
roll = [1, 1, 1, 1, 1, 1]
print("Maximum points:", calculate_points(roll))

In [ ]:
def count_permutations(roll, num_dice=6):
    np.array(roll).astype(int)
    counts = np.bincount(roll)

    numerator = math.perm(num_dice, num_dice)
    denominator = 1

    for i in counts:
        if i > 1:
            denominator *= math.factorial(i)


    return int(numerator / denominator)

roll = [1, 2, 3, 4, 5, 5]
count_permutations(roll)

## Simulations

In [ ]:
dice_dim = 6
num_dice = 2

rolls = combinations_with_replacement(list(range(1, dice_dim + 1)), num_dice)
num_rolls = int((np.math.factorial(num_dice + dice_dim - 1)) / 
                (np.math.factorial(num_dice) * np.math.factorial(dice_dim - 1)))

rolls_points_perms = np.zeros(shape=(num_rolls, num_dice + 2))
for i, r in enumerate(rolls):
    rolls_points_perms[i][:num_dice] = r
    rolls_points_perms[i][num_dice] = calculate_points(r)
    rolls_points_perms[i][num_dice + 1] = count_permutations(r, num_dice=num_dice)

np.savetxt("rolls.csv", rolls_points_perms, fmt='%d', delimiter=",")

In [ ]:
# Naive Expected Values
dice_dim = 6
num_dice = 6
for d in range(1, num_dice + 1):
    rolls = combinations_with_replacement(list(range(1, dice_dim + 1)), d)
    num_rolls = int((np.math.factorial(d + dice_dim - 1)) / 
                    (np.math.factorial(d) * np.math.factorial(dice_dim - 1)))
    total_possibilities = dice_dim**d

    expected_value = 0
    for i, r in enumerate(rolls):
        expected_value += calculate_points(r) * count_permutations(r, num_dice=d) / total_possibilities
    
    print(f'E({d}) = ' + str(expected_value))


In [ ]:
# Chance of Farkle
dice_dim = 6
num_dice = 6
for d in range(1, num_dice + 1):
    rolls = combinations_with_replacement(list(range(1, dice_dim + 1)), d)
    num_rolls = int((np.math.factorial(d + dice_dim - 1)) / 
                    (np.math.factorial(d) * np.math.factorial(dice_dim - 1)))
    total_possibilities = dice_dim**d

    total_farkles = 0
    for i, r in enumerate(rolls):
        if calculate_points(r) == 0:
            total_farkles += count_permutations(r, num_dice=d)
    
    print(f'P({d}) = ' + str(total_farkles / total_possibilities))